In [5]:
!pip install opencv-python
!pip install opencv-rolling-ball
!pip install sewar


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import pandas as pd
from skimage import io, filters, feature, measure, metrics
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import cv2
import os
from cv2_rolling_ball import subtract_background_rolling_ball
from sewar import full_ref
from skimage.metrics import structural_similarity as ssim

#Edge detection
from skimage.filters import roberts, sobel, scharr, prewitt, farid, threshold_multiotsu
from scipy import ndimage as nd
from skimage.filters.rank import entropy
from skimage.morphology import disk
import glob
import shutil
from skimage import io
from IPython.core.debugger import Pdb

In [2]:
df_batteryLidImages = pd.DataFrame(columns=['File_Name', 'SSIM', 'Expanded'])

In [3]:
def label_And_AppendBatteryLidImages(img_file, ref_img_file):
    global expanded, df_batteryLidImages
    
    ref_img = cv2.imread(ref_img_file, 1)
    img = cv2.imread(img_file, 1)

    ssim_skimg = ssim(ref_img, img,
                      data_range = img.max() - img.min(), 
                      multichannel = True)
    
    if expanded == 'Unexpanded' and ssim_skimg > 0.90:
        expanded = 'Expanded'
    elif expanded == 'Expanded' and ssim_skimg < 0.80:
        expanded = 'Unexpanded'
        
    img_file_name = os.path.splitext(os.path.basename(img_file))[0]
    
    print(img_file_name, ssim_skimg, expanded) 
    
    new_row = pd.DataFrame({'File_Name': [img_file_name], 'SSIM': [ssim_skimg], 'Expanded': [expanded]})
    
    df_batteryLidImages = pd.concat([df_batteryLidImages, new_row], ignore_index=True)
    
    shutil.move(f"D:\Volvo\BatteryLid Images Sealing\{img_file_name}.jpg", f"D:\Volvo\BatteryLid Images Sealing\{expanded}\{img_file_name}.jpg")
    shutil.move(f"D:\Volvo\BatteryLid Images Threshold Gaussian\{img_file_name}.jpg", f"D:\Volvo\BatteryLid Images Threshold Gaussian\{expanded}\{img_file_name}.jpg")
    shutil.move(f"D:\Volvo\BatteryLid Images Threshold Mean\{img_file_name}.jpg", f"D:\Volvo\BatteryLid Images Threshold Mean\{expanded}\{img_file_name}.jpg")
    shutil.move(f"D:\Volvo\BatteryLid Images Gabor\{img_file_name}.jpg", f"D:\Volvo\BatteryLid Images Gabor\{expanded}\{img_file_name}.jpg")
    shutil.move(f"D:\Volvo\BatteryLid Images Gabor Threshold Gaussian\{img_file_name}.jpg", f"D:\Volvo\BatteryLid Images Gabor Threshold Gaussian\{expanded}\{img_file_name}.jpg")
    shutil.move(f"D:\Volvo\BatteryLid Images Gabor Threshold Mean\{img_file_name}.jpg", f"D:\Volvo\BatteryLid Images Gabor Threshold Mean\{expanded}\{img_file_name}.jpg")
    
    #df_batteryLidImages = df_batteryLidImages.append(pd.DataFrame(, index=[0]), ignore_index=True)

In [8]:
image_files = glob.glob("D:\Volvo\BatteryLid Images Sealing\*.*")
image_files.sort(key=os.path.getmtime)

global expanded, df_batteryLidImages
expanded = 'Unexpanded'

for image_file in image_files:
    label_And_AppendBatteryLidImages(image_file, 'D:\\Volvo\\Images\\ExpandedSealing.jpg')

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_5004\1306436816.py:7: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_skimg = ssim(ref_img, img,


WIN_20240307_14_59_52_Pro_272_000001 0.7514254590872933 Unexpanded
WIN_20240307_14_59_52_Pro_273_000002 0.7468978703633852 Unexpanded
WIN_20240307_14_59_52_Pro_274_000003 0.7474010079107217 Unexpanded
WIN_20240307_14_59_52_Pro_275_000004 0.7474092172964669 Unexpanded
WIN_20240307_14_59_52_Pro_276_000005 0.7459050742466887 Unexpanded
WIN_20240307_14_59_52_Pro_277_000006 0.7435059943159796 Unexpanded
WIN_20240307_14_59_52_Pro_278_000007 0.7454827379732798 Unexpanded
WIN_20240307_14_59_52_Pro_279_000008 0.7449785074378457 Unexpanded
WIN_20240307_14_59_52_Pro_280_000009 0.7469759361688949 Unexpanded
WIN_20240307_14_59_52_Pro_281_000010 0.7452347161040631 Unexpanded
WIN_20240307_14_59_52_Pro_282_000011 0.7262918460718463 Unexpanded
WIN_20240307_14_59_52_Pro_283_000012 0.7286365355604939 Unexpanded
WIN_20240307_14_59_52_Pro_284_000013 0.7277227185956489 Unexpanded
WIN_20240307_14_59_52_Pro_285_000014 0.7281149327079901 Unexpanded
WIN_20240307_14_59_52_Pro_286_000015 0.7280311772638216 Unexpa

In [9]:
df_batteryLidImages.groupby('Expanded').count()

,File_Name,SSIM
Expanded,,
Expanded,9846,9846
Unexpanded,9691,9691


In [19]:
df_batteryLidImages.to_csv('D:\\Volvo\\BatteryLidImages.csv')